# Reinforcement Q-learning simple NN as function approximation

- portfolio grid of size 10 (0 to 1)
- 1 hidden neural layers 
- Improved by Dropout 

In [2]:
# Initialization
import numpy as np
import pandas as pd
import random
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline

import winsound

In [43]:
data = pd.read_csv('data.csv') #Three stocks (R,X_s,X_b,s_nom,s_pe,s_spr) 

In [3]:
data = pd.read_excel('sim_data_CER.xlsx') #Three stocks (R,X_s,X_b) Without predictors

# Initialization of the Tensorflow placeholders and the network

In [4]:
num_lags = 11
num_stocks=3
num_inputvar = (num_lags+1)*num_stocks
num_actions=10
hidden1 = 60
hidden2 = 45
hidden3 = 20

#Initialize Neural Network and set-up the placeholders
tf.reset_default_graph()
NN_input = tf.placeholder(shape=[1,num_inputvar],dtype=tf.float32)
w1 = tf.Variable(tf.random_uniform([num_inputvar,hidden1],0,1))
w2 = tf.Variable(tf.random_uniform([hidden1,hidden2],0,1))
w3 = tf.Variable(tf.random_uniform([hidden2,hidden3],0,1))
w4 = tf.Variable(tf.random_uniform([hidden3,num_actions],0,1))
b1 = tf.Variable(np.zeros((1,hidden1)), dtype=tf.float32)
b2 = tf.Variable(np.zeros((1,hidden2)), dtype=tf.float32)
b3 = tf.Variable(np.zeros((1,hidden3)), dtype=tf.float32)
b4 = tf.Variable(np.zeros((1,num_actions)), dtype=tf.float32)

h1 = tf.add(tf.matmul(NN_input,w1),b1)
h_drop1 = tf.nn.dropout(h1,0.5)
a1 = tf.nn.relu(h_drop1)

h2 = tf.add(tf.matmul(a1,w2),b2)
h_drop2 = tf.nn.dropout(h2,0.3)
a2 = tf.nn.relu(h_drop2)

h3 = tf.add(tf.matmul(a2,w3),b3)
h_drop3 = tf.nn.dropout(h3,0.1)
a3 = tf.nn.relu(h_drop3)

ol = tf.add(tf.matmul(a3,w4),b4)
output = tf.nn.softmax(ol)

A_Max = tf.argmax(output,1)
# Calculate loss for the NN from the Q values
Q_Next = tf.placeholder(shape=[1,num_actions],dtype=tf.float32)
diff = tf.subtract(Q_Next,output)
loss = tf.reduce_sum(tf.multiply(diff,diff))
learning_rate = tf.placeholder(tf.float32, shape=[])
# cross_entropy = tf.reduce_mean(-tf.reduce_sum(a4 * tf.log(Q_Next), reduction_indices=[1]))
optmzr = tf.train.GradientDescentOptimizer(learning_rate = learning_rate).minimize(loss)
A = np.linspace(0,1,num_actions) # portfolio weights of stocks (1-weight) is the weight in the bonds

# Training of the NN function approximation

In [ ]:
# initialize
init = tf.global_variables_initializer()
startepsilon = 0.8
endepsilon = 0.01
startlr = 0.8
endlr = 0.01
startgamma = 0.01
endgamma = 0.8
TWlist = []
Index = []
MWeights = []
Turnover = []
chooserlist = []
RU = []
learningrateQ = 0.8
n = 707
periods = 2
epochs = 50     # preferred to have a low amount of epochs because otherwise the the same data is used multiple times (usually not the case in stock returns)

# data parsing
dates = data['Date']
mdata = data[['r','xs','xb']]
mdata.index = pd.DatetimeIndex(dates)

# include the lags
for i in range(1,num_lags+1):  # 1 till 11
    mdata['r_lag' + str(i)] = mdata['r'].shift(i)
    mdata['xs_lag' + str(i)] = mdata['xs'].shift(i)
    mdata['xb_lag' + str(i)] = mdata['xb'].shift(i)

mdata.drop(mdata.head(num_lags).index, inplace=True)
# have a look at the data (if not include NANs)
# print(mdata)

# Train the the Q-function DNN
with tf.Session() as sess:
    for i in range(241,int(n-periods-1)):
        lostlist = []
        OptimalWeights = np.zeros(periods-1)
        currentK = 0
        print(i)
        rstd = 1
        rmean = 0
        
        while currentK < periods - 1:
            epsilon = 0.15
            lr = 0.00003
            gamma = 0.5
            sess.run(init) # initialize the Neural Network again
            currentEpoch = 0
            rlist = []               
            indexes = np.asarray(range(i+currentK-1)) 
            
            while currentEpoch < epochs:
#                 if(currentEpoch > 0):
#                     epsilon -= (startepsilon-endepsilon)/epochs
#                     lr -= (startlr-endlr)/epochs
#                     gamma += (startgamma-endgamma)/epochs
                    
                #Training of the Q-Network for the data available (with Neural Nets) 
                for j in indexes:
                    s = mdata.iloc[j,:].values.reshape(1,num_inputvar)
                    #Choose an action by greedily (with e chance of random action) from the Q-network
                    a_int,allQ = sess.run([A_Max,output],feed_dict={NN_input:s})
                    a = A[a_int-1]  # -1 because index
                    if np.random.rand(1) < epsilon:
                        a = random.choice(A)

                    #Get new state and reward from environment
                    s1 = mdata.iloc[j+1,:].values.reshape(1,num_inputvar)
                    r = (a*s1[0][1] + (1-a)*s1[0][2]) #reward: this is now the wealth gained from this step, but could be other rewards like utility
                    rlist.append(r)
                    Q = sess.run(output,feed_dict={NN_input:s1})
                    
                    #Obtain maxQ' and set our target value for chosen action.
                    Q1 = np.max(Q)
                    targetQ = allQ
                    if(len(rlist)>1):
                        rstd = np.std(rlist)
                        rmean = np.mean(rlist)
                    targetQ[0,int(a*(num_actions-1))] = targetQ[0,int(a*(num_actions-1))] + learningrateQ*((r-rmean)/rstd + gamma*Q1 - targetQ[0,int(a*(num_actions-1))])
#                     targetQ[0,int(a*(num_actions-1))] = targetQ[0,int(a*(num_actions-1))] + learningrateQ*(r + gamma*Q1 - targetQ[0,int(a*(num_actions-1))]) 
#                     targetQ[0,int(a*(num_actions-1))] = r + gamma*Q1
#                     rlist.append(r)
                    #Train the neural network using target and predicted Q values
                    opt,W1,loss_ = sess.run([optmzr,w1,loss],feed_dict={NN_input:s,Q_Next:targetQ,learning_rate:lr})
                    lostlist.append(loss_)
                currentEpoch += 1
            
            # After training now calculate the optimal weights for the K=60 periods to come
            s = mdata.iloc[i+currentK,:].values.reshape(1,num_inputvar)
            a_int,allQ = sess.run([A_Max,output],feed_dict={NN_input:s})
            aOpt = A[a_int-1]
            OptimalWeights[currentK] = aOpt
            currentK += 1
            
        # For insight purposes (Write away)
        plt.ion()
        plt.plot(lostlist)
        plt.show()
        plt.plot(rlist)
        plt.show()
        
        firstdiff = OptimalWeights[1:] - OptimalWeights[:-1]
        MWeights.append(np.mean(OptimalWeights))
        TerminalWealth = np.exp(sum(OptimalWeights*mdata[i+1:i+currentK+1]['xs'] + (1-OptimalWeights)*mdata[i+1:i+currentK+1]['xb']))
        TWlist.append(TerminalWealth)
        Index.append(i)
        Turnover.append(sum(abs(firstdiff*np.exp(mdata[i+1:i+currentK]['xs'])) + abs((1-firstdiff)*np.exp(mdata[i+1:i+currentK]['xb']))))
        RU.append((1/(1-5))*pow(TerminalWealth,(1-5)))
        print(OptimalWeights)
        print(TerminalWealth)
        
        # Make a sound when done
#         duration = 1000  # millisecond
#         freq = 440  # Hz
#         winsound.Beep(freq, duration)
        
        print('Writing away results')
        df = pd.DataFrame({'index date':Index,'TW':TWlist, 'Mean Weights Xs':MWeights,'Turnover':Turnover, 'Realized Utility':RU})
        df.to_excel('DNN_2.xlsx', sheet_name='sheet1', index=False)
print('Done!')
        

241
